In [2]:
from typing import *
import wandb
import numpy as np

import llminference as L

api = wandb.Api()
def get_run(id: str) -> wandb.apis.public.Run:
    return api.run(f"research/sparse-attention/runs/{id}")

In [7]:
# From Figures.ipynb
run_ids = {'cnn_dailymail': ['vwnh8cu6', '539f9rom', '82e0zahq', 'lbpj144w'],
 'repetition': ['wez29r4a', '3myiq6s1', 'tyhsk7u3', '4kuqkuvs'],
 'squad': ['u65h4c6x', 's73en2vc', 'tiv7kxft', 'rhc29d4t'],
 'triviaqa': ['cjcx2ccn', 'h08zynp0', 's3har2ni', 'i1klec42'],
 'wikitext_bpc': ['xxlf2lqq', 'z1fclt4r', 'nwepshqs', 'zelxigof']}

display_name = dict(
    dense="DENSE",
    ann="SPARQ",
    eviction="H2O",
    local="LOCAL",
)

for _, ids in run_ids.items():
    ids.sort(key=lambda id: list(display_name.keys()).index(get_run(id).config["sparsity"]["name"]))

In [10]:
def shorten(text: str, length: int) -> str:
    return text if len(text) <= length else text[:length] + "..."

def show_example(example: Dict[str, Any], task: str):
    results = []
    for id in run_ids[task]:
        m = [r for r in get_run(id).summary.results if r["id"] == example["id"]]
        if len(m) > 2 if task == "triviaqa" else len(m) != 1:
            raise ValueError(f"Multiple matching results for {example['id']}")
        results.append(m[0])

    key = dict(squad="match", triviaqa="match", cnn_dailymail="rougeL", wikitext_bpc="bpc", repetition="match_length_char")[task]
    print({get_run(id).config["sparsity"]["name"]: r[key] for id, r in zip(run_ids[task], results)})
    print()
    print(f"### PROMPT ({example['id']})")
    if task == "cnn_dailymail":
        print(shorten(example["context"], 80) + L.summarisation.DEFAULT_PROMPT)
    elif task == "repetition":
        offset = example["context"].index(example["prompt"])
        lines = example["context"].split("\n")
        line_offset = np.cumsum([len(line) + 1 for line in lines])
        line_index = (line_offset < offset).sum()
        print("\n".join(lines[:2] + ["...", "..."] + lines[line_index:][:10] + ["...", "..."])
              + L.repetition.PROMPT_PREFIX + example["prompt"])
    elif task == "triviaqa":
        lines = example["context"].replace("\n\n", "\n").split("\n")
        print("\n".join(shorten(s, 50) for s in lines[:5] + ["...", "..."] + lines[-5:]))
        print(example["prompt"])
    elif task == "squad":
        lines = example["context"].split("\n")
        print("\n".join(shorten(s, 50) for s in lines[:-1]) + "\n" + lines[-1])
        print(example["prompt"])
    print()
    print("### OUTPUT")
    for id, result in zip(run_ids[task], results):
        out = shorten(result["output"].split("\n")[0], 80)
        print(f'{display_name[get_run(id).config["sparsity"]["name"]]:>5}: {out}')


# SQuAD

In [11]:
index = 18
show_example(
    L.qa.add_few_shot_prompt(L.qa.SQuAD.data()[index], k=1, prompt_template=L.qa.get_default_prompt_template("meta-llama/Llama-2-7b-hf", shots=1)),
    "squad",
)

Found cached dataset squad (/nethome/douglaso/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached processed dataset at /nethome/douglaso/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-03400b5d173ab796.arrow
Loading cached shuffled indices for dataset at /nethome/douglaso/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-c374d8616db3e9e0.arrow


{'dense': True, 'ann': True, 'eviction': False, 'local': False}

### PROMPT (56de0ed14396321400ee257b)
Title: Private school. Background: In many parts o...
Title: University of Chicago. Background: The Univ...
Title: Nikola Tesla. Background: Near the end of h...
Title: Ctenophora. Background: Since all modern ct...
Title: Chloroplast. Background: Lepidodinium virid...
Title: Construction. Background: Construction proj...
Title: Normans. Background: Institutionally, the N...
Title: Genghis Khan. Background: At this time, non...
From what you've just read about Normans, please answer the following questions.
Question: What does Kitab Rudjdjar mean in English?
Answer: The Book of Roger
Question: What kind of bureaucracy did the Normans institute?
Answer:

### OUTPUT
DENSE: Meritocratic
SPARQ: Meritocratic
  H2O: Mer Mer Mer Mer Mer Mer Mer Mer Mer
LOCAL: Feudalism


# TriviaQA

In [12]:
index = 9
example = L.qa.add_few_shot_prompt(L.qa.TriviaQA.data(context="wiki")[index], k=0, prompt_template=L.qa.get_default_prompt_template("meta-llama/Llama-2-7b-hf", shots=0))
show_example(example, "triviaqa")

Found cached dataset trivia_qa (/nethome/douglaso/.cache/huggingface/datasets/trivia_qa/rc/1.2.0/ee76d8a9403e71177e2a3fa7e414d1ee28a79a0970d9176f62f268798aa64b31)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /nethome/douglaso/.cache/huggingface/datasets/trivia_qa/rc/1.2.0/ee76d8a9403e71177e2a3fa7e414d1ee28a79a0970d9176f62f268798aa64b31/cache-a997c05712301d33.arrow
Loading cached shuffled indices for dataset at /nethome/douglaso/.cache/huggingface/datasets/trivia_qa/rc/1.2.0/ee76d8a9403e71177e2a3fa7e414d1ee28a79a0970d9176f62f268798aa64b31/cache-49a4e46d67c5b9d0.arrow


{'dense': True, 'ann': True, 'eviction': True, 'local': False}

### PROMPT (dpql_5685)
Apéritifs and digestifs ( and) are drinks, typical...
Apéritifs
An apéritif is an alcoholic beverage usually serve...
"Apéritif" may also refer to a snack that precedes...
"Apéritif" is a French word derived from the Latin...
...
...
* Distilled liquors (ouzo, tequila, whisky or akva...
* Liquor cocktails (Black Russian, Rusty Nail, etc...
In certain areas, it is not uncommon for a digesti...
Bitter digestifs typically contain carminative her...
In many countries, people drink alcoholic beverage...
Question: Which aperitif is named for the Paris chemist who created it in 1846?
Answer:

### OUTPUT
DENSE: Dubonnet
SPARQ: Dubonnet
  H2O: Dubonnet
LOCAL: Vermouth.


# CNN/DailyMail

In [13]:
index = 5
show_example(L.summarisation.CnnDailymail.data()[index], "cnn_dailymail")

Found cached dataset cnn_dailymail (/nethome/douglaso/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /nethome/douglaso/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de/cache-14745bd540514b27.arrow
Loading cached shuffled indices for dataset at /nethome/douglaso/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de/cache-e1812a493f78a625.arrow


{'dense': 0.15172413793103448, 'ann': 0.15894039735099338, 'eviction': 0.19736842105263155, 'local': 0.09599999999999999}

### PROMPT (417e9c57edd888a811e860429219b3a66f16d92c)
Article: Being at Old Trafford on Saturday was a surreal experience – and I’m no...
Summary:

### OUTPUT
DENSE: Manchester United fans have been left frustrated by the club's lack of attacking...
SPARQ: Manchester United fans have been left frustrated by the team's performances this...
  H2O: Manchester United fans are not happy with the team's performance this season. Th...
LOCAL: The rebuild is underway. But it’s unfair to expect 'Fergie football' so soon.


# Repetition

In [14]:
index = 0
show_example(L.repetition.Shakespeare.data()[index], "repetition")

Found cached dataset tiny_shakespeare (/nethome/douglaso/.cache/huggingface/datasets/tiny_shakespeare/default/1.0.0/b5b13969f09fe8707337f6cb296314fbe06960bd9a868dca39e713e163d27b5e)


  0%|          | 0/3 [00:00<?, ?it/s]

{'dense': 258, 'ann': 254, 'eviction': 5, 'local': 0}

### PROMPT (5049)
First Citizen:
Before we proceed any further, hear me speak.
...
...
Our steed the leg, the tongue our trumpeter.
With other muniments and petty helps
In this our fabric, if that they--

MENENIUS:
What then?
'Fore me, this fellow speaks! What then? what then?

First Citizen:
Should by the cormorant belly be restrain'd,
...
...
 the tongue our trumpeter.
With other muniments and petty helps
In this our fabric, if that they--

MENENIUS:
What then?
'Fore me,

### OUTPUT
DENSE: this fellow speaks! What then? what then?
SPARQ: this fellow speaks! What then? what then?
  H2O: this is most
LOCAL: I cannot tell.


# WikiText BPC

In [16]:
index = 0
example = L.bpc.WikiText.data()[index]
lines = example["prefill"].split("\n")
print(f"\\begin{{verbatim}}### QUERY ({index})")
print()
print("\n".join([shorten(line, 80) for line in lines[:-1]]))
print("\\end{verbatim}")
print("\\" + lines[-1] + "{\color{blue}\\" + example["reference"] + "}")
print("\\begin{verbatim}### BPC")
for id in run_ids["wikitext_bpc"]:
    run = get_run(id)
    bpc = run.summary.results[index]["bpc"]
    print(f'{display_name[get_run(id).config["sparsity"]["name"]]:>5}: {bpc:.3f}')
print("\\end{verbatim}")

Found cached dataset wikitext_document_level (/nethome/douglaso/.cache/huggingface/datasets/EleutherAI___wikitext_document_level/wikitext-103-raw-v1/1.0.0/c7f10a7786444f898dd236db33d4bee9b130f8cbcac690e7bde9b0d027e19fc1)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /nethome/douglaso/.cache/huggingface/datasets/EleutherAI___wikitext_document_level/wikitext-103-raw-v1/1.0.0/c7f10a7786444f898dd236db33d4bee9b130f8cbcac690e7bde9b0d027e19fc1/cache-0153ebe9c9b2083a.arrow
Loading cached shuffled indices for dataset at /nethome/douglaso/.cache/huggingface/datasets/EleutherAI___wikitext_document_level/wikitext-103-raw-v1/1.0.0/c7f10a7786444f898dd236db33d4bee9b130f8cbcac690e7bde9b0d027e19fc1/cache-9e9e22ca5a4dc438.arrow


\begin{verbatim}### QUERY (0)

 = County Road 595 ( Marquette County , Michigan ) = 
 
 County Road 595 ( CR 595 , Co . Rd . 595 ) is a proposed primary county road in...
 In 2003 , a flood along the Dead River destroyed or forced the closure of sever...
 The Marquette County Road Commission ( MCRC ) applied for permits from the stat...
 
 = = Route description = = 
 
 The county road would start at its southern end at an intersection with US 41 /...
 
 = = History = = 
 
 
 = = = Background = = = 
 
 On May 14 , 2003 , a section of the Silver Lake Dam failed in northwestern Marq...
\end{verbatim}
\ Since the flood a new , roughly 100 @-@ foot @-@ tall ( 30 m ) bridge{\color{blue}\ has been constructed on CR 510 , ensuring that any future flooding on the Dead River would not necessitate closure of the modern bridge . This structure was built at that height in order to benefit commercial interests in the area , as well as to provide the area with reliable public and emergency access . 
